## Загрузка данных и инициализация модели

In [1]:
# https://drive.google.com/file/d/1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG/view?usp=sharing # ссылка для ручной загрузки данных

In [4]:
!pip install gdown

In [5]:
!gdown 1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG && tar -xf kaggle.tar

Downloading...
From (original): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG
From (redirected): https://drive.google.com/uc?id=1Wd5awSAMCNKlieCv5xnSXy_we7zF9NsG&confirm=t&uuid=6f032f7c-0696-427f-b908-f30d2b81ee5c
To: /kaggle/working/kaggle.tar
100%|████████████████████████████████████████| 138M/138M [00:03<00:00, 35.0MB/s]


In [6]:
!ls -l kaggle/wav | wc -l

643


Для базового решения задания воспользуемся предобученной моделью Whisper от OpenAI. Для этого обратимся к платформе [HuggingFace](https://huggingface.co/collections/openai/whisper-release-6501bba2cf999715fd953013)

In [7]:
import os
import soundfile as sf
import torch
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import string

In [8]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from torch.nn.parallel import DataParallel
device = torch.device("cuda:0")
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
processor.feature_extractor.return_attention_mask = True
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)


preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

## Baseline подход к инференсу модели

In [9]:
def batch_inference(model, processor, path_to_wavs, batch_size, sampling_rate=16000):
    results = {}
    wav_files = os.listdir(path_to_wavs)

    forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe") # generates task specific special tokens
    results = {}
    for i in tqdm(range(0, len(wav_files), batch_size), total=np.ceil(len(wav_files) / batch_size)):
        audio_paths = wav_files[i : i + batch_size]

        batch = []

        for path in audio_paths:
            audio, _ = sf.read(os.path.join(path_to_wavs, path))
            batch.append(audio)
        inputs = processor(batch, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

        x, x_masks = inputs["input_features"].to(device), inputs["attention_mask"].to(device)

        with torch.no_grad():
            model.eval()
            output_ids = model.generate(x, forced_decoder_ids=forced_decoder_ids, attention_mask=x_masks,num_beams = 2, num_return_sequences = 2, repetition_penalty=1.5, return_dict = True)

        transcribtion = processor.batch_decode(output_ids, skip_special_tokens=True)
        transcribtion = iter(transcribtion)
        
        for path in audio_paths:
            for j in range(2):
                results.setdefault(path, []).append(next(transcribtion))


        
        
    return results

In [10]:
path_to_wavs = "kaggle/wav"
batch_size = 3

In [11]:
results = batch_inference(model=model, processor=processor, path_to_wavs=path_to_wavs, batch_size=batch_size)

100%|██████████| 214/214.0 [25:06<00:00,  7.04s/it]


In [12]:
results 

{'audio_288.wav': [' Вы как там?', ' Вы как там?'],
 'audio_522.wav': [' Меня зовут Анна Рязанова, я из Санкт-Петербурга, мне 36.',
  ' Меня зовут Анна Рязанова, я из Санкт-Петербурга, мне 36.'],
 'audio_292.wav': [' Информационная безопасность. Мы готовимся к поступлению в этот ВУЗ. Я учусь на 42-й кафедре информационной безопасности. Почему ты выбрала именно МИФИ?',
  ' Информационная безопасность. Мы готовимся к поступлению в этот ВУЗ. Я учусь на 42-й кафедре информационной безопасности. Почему ты выбрала именно МИФИ?'],
 'audio_355.wav': [' Принято считать, что о благих делах не говорят. Но если о них не говорить, то...',
  ' Принято считать, что о благих делах не говорят. Но если о них не говорить, то...'],
 'audio_411.wav': [' Меня в интернете действительно знают как Вилсаком, но на самом деле я повелитель квантовых точек и, конечно, магистр распаковок. Поэтому сегодня...',
  ' Меня в интернете действительно знают как Вилсаком, но на самом деле я повелитель квантовых точек и, кон

## Отранжируем предложенные варианты с помощью BERT

In [15]:
import torch

from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Определите функцию для оценки уверенности BERT
def evaluate_confidence(text):
    inputs = tokenizer.encode_plus(text, 
                                    add_special_tokens=True, 
                                    max_length=512, 
                                    return_attention_mask=True, 
                                    return_tensors='pt')
    outputs = model(inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))
    confidence = torch.nn.functional.softmax(outputs.last_hidden_state[:, 0, :], dim=1)
    return confidence.max().item()

# Пройдите по словарю и оставьте только одно предложение в каждом ключе
for key, values in results.items():
    confidences = [evaluate_confidence(value) for value in values]
    best_index = confidences.index(max(confidences))
    results[key] = [values[best_index]]


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=Tr

In [ ]:
results

In [16]:
!pip install num2words

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 8.6 MB/s eta 0:00:00


In [17]:
import num2words

# Функция для замены цифр на слова
def replace_numbers_with_words(text):
    import re
    numbers = re.findall(r'\d+', text)
    for number in numbers:
        text = text.replace(number, num2words.num2words(int(number), lang='ru'))
    return text

# Пройдите по словарю и замените цифры на слова
for key, values in results.items():
    new_values = []
    for value in values:
        new_value = replace_numbers_with_words(value)
        new_values.append(new_value)
    results[key] = new_values


In [18]:
results

{'audio_288.wav': [' Вы как там?'],
 'audio_522.wav': [' Меня зовут Анна Рязанова, я из Санкт-Петербурга, мне тридцать шесть.'],
 'audio_292.wav': [' Информационная безопасность. Мы готовимся к поступлению в этот ВУЗ. Я учусь на сорок два-й кафедре информационной безопасности. Почему ты выбрала именно МИФИ?'],
 'audio_355.wav': [' Принято считать, что о благих делах не говорят. Но если о них не говорить, то...'],
 'audio_411.wav': [' Меня в интернете действительно знают как Вилсаком, но на самом деле я повелитель квантовых точек и, конечно, магистр распаковок. Поэтому сегодня...'],
 'audio_198.wav': [' Ему еще предстоит удержать власть в своих руках. Если выживет, конечно. Александр начинает править в Новгороде в шестнадцать лет.'],
 'audio_620.wav': [' Как будто там же, ну да. Вот так вот.'],
 'audio_376.wav': [' Номер пять. Вы клоун из барахолки, точно не Ачетыре. Вставай, клоун, иди домой.'],
 'audio_52.wav': [' Человек прожил двадцать два часа в пасти кита. Ого! В общем, ты уволен.

## Sample submission
Подготовим пример для загрузки на платформу

In [19]:
import string

def dummy_postprocessing(data):
    for filename, hypo in data.items():
        hypo = hypo[0].strip()  # берем первый элемент списка
        hypo = hypo.translate(str.maketrans('', '', string.punctuation))
        # hypo = hypo.replace('[', '').replace(']', '')  # удаляем скобки
        hypo = hypo.lower()
        if not hypo:
            data[filename] = "-"
        else:
            data[filename] = hypo  # сохраняем результат в виде списка
    return data

clean_data = dummy_postprocessing(results)


In [20]:
sample_submission = pd.read_csv("kaggle/sample_submission.csv")

In [21]:
sample_submission.head(20)

,filename
0,audio_0.wav
1,audio_1.wav
2,audio_2.wav
3,audio_3.wav
4,audio_4.wav
5,audio_5.wav
6,audio_6.wav
7,audio_7.wav
8,audio_8.wav
9,audio_9.wav


In [22]:
sample_submission["hypo"] = sample_submission["filename"].apply(lambda x: clean_data[x])

In [23]:
sample_submission.insert(0, "id", sample_submission.index)

In [24]:
sample_submission.head(10)

,id,filename,hypo
0,0,audio_0.wav,продолжение следует
1,1,audio_1.wav,субтитры создавал dimatorzok
2,2,audio_2.wav,потише жрем враг на гремле сюда идти я сказал
3,3,audio_3.wav,слышали не понял
4,4,audio_4.wav,быстро за мной


In [25]:
sample_submission.to_csv("baseline.csv", index=False)

## Идеи по улучшению предсказаний:
* Всегда ли LAS архитектура работает на тишине корректно?
* Как насчет нормализации текста? (19.02 - девятнадцатое февраля, 20 лет - двадцать лет)
* Может быть попробовать рескоринг? (возвращаем больше, чем топ-1 гипотезу из бима)
* Может быть взять модель побольше?
